In [3]:
from utils import extract
from ned import ned

In [2]:
paragraph = "前「香港眾志」成員周庭8月10日被港警以涉嫌違反港區國安法逮捕，後獲准保釋。周庭獲釋後昨天首度向警方報到，她表示，港警指香港眾志去年在日經新聞刊廣告，涉違反國安法。"

## 摘要

* 產生候選片語
* ckip + 實體名詞辨識
* ngram + 實體名詞辨識
* 聯結次數

### 透過維基百科產生候選片語

* 無歧異詞語
* 有歧異詞語，常見用法只有一種。
* 有歧異詞語，無法辨識。

In [8]:


def convert(obj):
    if isinstance(obj, dict):
        return {ned._id2title(key): value for key, value in obj.items()}
    elif isinstance(obj, dict):
        return [ned._id2title(key) for key in obj]
    elif isinstance(obj, int):
        return ned._id2title(obj)

def simple_coverter(data):
    
    return {key: convert(value) for key, value in data.items()}


In [14]:
print(simple_coverter(ned.gen_NE_candidates("牛奶針")))

NameError: name 'ned' is not defined

### 無歧異詞語

* 牛奶針 -> 异丙酚

In [11]:
print(simple_coverter(ned.gen_NE_candidates("陳時中")))

{'title': '', 'redirect': '', 'bold': None, 'anchor': {'陳時中 (政治人物)': 10}}


### 有歧異詞語，但常見用法只有一種。

* 陳時中 -> 陳時中 (政治人物)

In [15]:
print(simple_coverter(ned.gen_NE_candidates("柯瑞")))

{'title': '', 'redirect': '', 'bold': None, 'anchor': {'柯瑞·巴斯特': 1}}


### 有歧異詞語，無法確認是誰

* 柯瑞 -> ???

#### 聯合報 2019術語

* 約3成可以在維基百科找到
* 醫療 金融 保險用語居多

In [18]:
! head -n 40 2019_udn_in_wiki.tsv

負所得稅制		
屏下指紋辨識		
換匯交易		
城市貨幣		
備轉容量率		
心室頻脈		
稅收超徵		
歲計賸餘		
北斗衛星系統		
OBU	OBU	https://zh.wikipedia.org/wiki/OBU
DBU	1,8-二氮杂二环［5.4.0］十一碳-7-烯	https://zh.wikipedia.org/wiki/1,8-%E4%BA%8C%E6%B0%AE%E6%9D%82%E4%BA%8C%E7%8E%AF%EF%BC%BB5.4.0%EF%BC%BD%E5%8D%81%E4%B8%80%E7%A2%B3-7-%E7%83%AF
髖部骨折	髖部骨折	https://zh.wikipedia.org/wiki/%E9%AB%96%E9%83%A8%E9%AA%A8%E6%8A%98
閨密節		
急性膽囊炎		
修昔底德陷阱	修昔底德陷阱	https://zh.wikipedia.org/wiki/%E4%BF%AE%E6%98%94%E5%BA%95%E5%BE%B7%E9%99%B7%E9%98%B1
O-RAN聯盟		
Thunderbolt	Thunderbolt	https://zh.wikipedia.org/wiki/Thunderbolt
GDPR	歐盟一般資料保護規範	https://zh.wikipedia.org/wiki/%E6%AD%90%E7%9B%9F%E4%B8%80%E8%88%AC%E8%B3%87%E6%96%99%E4%BF%9D%E8%AD%B7%E8%A6%8F%E7%AF%84
腰椎椎間盤突出		
錐體外症候群	錐體外症候群	https://zh.wikipedia.org/wiki/%E9%8C%90%E9%AB%94%E5%A4%96%E7%97%87%E5%80%99%E7%BE%A4
胃食道逆流	胃食道逆流	https://zh.wikipedia.org/wiki/%E8%83%83%E9%A3%9F%E9%81%93%E9%80%86%E6%B5%81
颮線	飑线	https://zh.wikipedia.org/wiki/%E9%A3%91%E7%BA%BF
外溢保單		
紅籌架構		
類載板		
邊緣運算	邊緣運算	https://zh.wikipedia.org/wiki/%E9%

### ckip + 實體名詞辨識

In [19]:
from ner import ckip_ner

In [22]:
print(paragraph)

print(ckip_ner.gen_ner(paragraph))

前「香港眾志」成員周庭8月10日被港警以涉嫌違反港區國安法逮捕，後獲准保釋。周庭獲釋後昨天首度向警方報到，她表示，港警指香港眾志去年在日經新聞刊廣告，涉違反國安法。
[(2, 6, 'ORG', '香港眾志'), (9, 11, 'PERSON', '周庭'), (38, 40, 'PERSON', '周庭'), (57, 58, 'GPE', '港'), (62, 64, 'PERSON', '眾志'), (67, 71, 'WORK_OF_ART', '日經新聞'), (78, 81, 'LAW', '國安法')]


In [26]:
for start, end, _type, text in ckip_ner.gen_ner(paragraph):
    wiki_title = ned.disambiguate_NE(text)
    print(text, "->", wiki_title)

香港眾志 -> 香港眾志
周庭 -> 周庭
周庭 -> 周庭
港 -> 香港
眾志 -> 香港眾志
日經新聞 -> 日本經濟新聞
國安法 -> 


### bigram~five gram + 實體名詞辨識

In [28]:
from utils import extract

In [35]:
res = []
for idx, text in extract.split_string(paragraph):
    for start, end in extract.gen_ngram(text):
        
        wiki_title = ned.disambiguate_NE(text[start:end])
        if wiki_title:
            res.append((idx+start ,text[start: end], wiki_title))
res = extract.merge_span(res)
print(paragraph)

for start, text, wiki_title in res:
    print(start, text, "->", wiki_title)

前「香港眾志」成員周庭8月10日被港警以涉嫌違反港區國安法逮捕，後獲准保釋。周庭獲釋後昨天首度向警方報到，她表示，港警指香港眾志去年在日經新聞刊廣告，涉違反國安法。
2 香港眾志 -> 香港眾志
9 周庭 -> 周庭
11 8月10日 -> 8月10日
17 港警 -> 香港警務處
20 涉嫌 -> 犯罪嫌疑人
24 港區國安法 -> 中華人民共和國香港特別行政區維護國家安全法
29 逮捕 -> 逮捕
35 保釋 -> 保釋
38 周庭 -> 周庭
43 昨天 -> 昨天
48 警方 -> 警隊
54 表示 -> 群表示論
57 港警 -> 香港警務處
60 香港眾志 -> 香港眾志
67 日經新聞 -> 日本經濟新聞
72 廣告 -> 廣告


### 聯結次數 (可當作重要性過濾)

In [36]:
wiki_titles = {title for _, _, title in res}

In [37]:
print(wiki_titles)

{'香港眾志', '犯罪嫌疑人', '昨天', '群表示論', '日本經濟新聞', '警隊', '8月10日', '中華人民共和國香港特別行政區維護國家安全法', '周庭', '保釋', '香港警務處', '逮捕', '廣告'}


In [45]:
from collections import Counter
title_count = Counter()

for anchor_count in ned.load_anchor.ANCHOR_TABLE.anchor2count.values():
    for title, count in anchor_count.items():
        title_count[ned._id2title(title)] += count

In [46]:
title_record = []
for title in wiki_titles:
    title_record.append((title, title_count[title]))
    print(title, "->", title_count[title])

香港眾志 -> 121
犯罪嫌疑人 -> 171
昨天 -> 4
群表示論 -> 85
日本經濟新聞 -> 92
警隊 -> 129
8月10日 -> 10
中華人民共和國香港特別行政區維護國家安全法 -> 77
周庭 -> 35
保釋 -> 150
香港警務處 -> 1291
逮捕 -> 334
廣告 -> 1244


In [48]:
title_record.sort(key=lambda x: x[1])

In [49]:
print(title_record)

[('昨天', 4), ('8月10日', 10), ('周庭', 35), ('中華人民共和國香港特別行政區維護國家安全法', 77), ('群表示論', 85), ('日本經濟新聞', 92), ('香港眾志', 121), ('警隊', 129), ('保釋', 150), ('犯罪嫌疑人', 171), ('逮捕', 334), ('廣告', 1244), ('香港警務處', 1291)]


### 觀察

* 時間類別次數較低，可食用ner去除(ex. 昨天, 8月10日)
* 次數過高的往往不重要 ('香港警務處': 1291, '廣告': 1244)

In [50]:
terms = ["修昔底德陷阱", "O-RAN聯盟", "Thunderbolt", "歐盟一般資料保護規範", "腰椎椎間盤突出", "錐體外症候群", "胃食道逆流"]

In [51]:
for term in terms:
    print(term, "->", title_count[term])

修昔底德陷阱 -> 3
O-RAN聯盟 -> 0
Thunderbolt -> 14
歐盟一般資料保護規範 -> 6
腰椎椎間盤突出 -> 0
錐體外症候群 -> 12
胃食道逆流 -> 35


### 觀察

* 術語次數比較低
* 可考慮計算維基百科該詞出現且被連結的次數比上該詞出現次數